In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path 
import socket
#from rich import inspect

Set data path

In [11]:
this_computer_name = socket.gethostname()
print(this_computer_name)

DESKTOP-SGOMPNG


In [12]:

if this_computer_name == 'DESKTOP-V3D343U':
    freemocap_validation_data_path = Path(r"I:\My Drive\HuMoN_Research_Lab\FreeMoCap_Stuff\FreeMoCap_Balance_Validation\data")
else:
    freemocap_validation_data_path = Path(r"C:\Users\kiley\Documents\HumonLab\SampleFMC_Data\FreeMocap_Data-20220216T173514Z-001\FreeMocap_Data")

In [13]:
## Build'n'Load Stuff (paths and arrays)
#freemocap_validation_data_path = Path(r"C:\Users\aaron\Documents\HumonLab\Spring2022\ValidationStudy\FreeMocap_Data") #path to your data folder





sessionID = 'sesh_2022-02-10_16_24_00' #name of the sessionID folder
this_freemocap_session_path = freemocap_validation_data_path / sessionID
this_freemocap_data_path = this_freemocap_session_path/'DataArrays'
mediapipe_data_path = this_freemocap_data_path / 'mediaPipeSkel_3d_smoothed.npy'


In [14]:

mediapipeSkel_fr_mar_dim = np.load(mediapipe_data_path) #loads in the data as a numpy array

In [21]:
num_pose_joints = 33 #number of pose joints tracked by mediapipe 




pose_joint_range = range(num_pose_joints)
#frame_range = range(first_frame,last_frame)

mediapipe_pose_data = mediapipeSkel_fr_mar_dim[:,0:num_pose_joints,:] #load just the pose joints into a data array, removing hands and face data 

num_frames = len(mediapipe_pose_data)

num_frame_range = range(num_frames)

skel_x = mediapipe_pose_data[:,:,0]
skel_y = mediapipe_pose_data[:,:,1]
skel_z = mediapipe_pose_data[:,:,2]

![](WinterAnthropometryTable.jpeg?raw=true)

In [17]:

#segments is a list of the body segments that we want the center of mass for. joint_connections is the corresponding joints that make up that segment in order of [proximal joint,distal joint]
# segment_COM_lengths is a corresponding list of center of mass lengths for each segment, and segment_COM_percentages is a corresponding list of the percentage of total body COM each segment takes up. 
segments = [
'head',
'trunk',
'right_upper_arm',
'left_upper_arm',
'right_forearm',
'left_forearm',
'right_hand',
'left_hand',
'right_thigh',
'left_thigh',
'right_shin',
'left_shin',
'right_foot',
'left_foot'
]

joint_connections = [
['right_ear', 'left_ear'],
['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip'], #do these joint_connections have to correspond to mediapipe? yes
['right_shoulder','right_elbow'],
['left_shoulder','left_elbow'],
['right_elbow', 'right_wrist'],
['left_elbow', 'left_wrist'],
['right_wrist', 'right_index', 'right_pinky'], #need to spend some time on the hands,can there be more than two elements in these lists?
['left_wrist', 'left_index', 'left_pinky'],
['right_hip', 'right_knee'],
['left_hip', 'left_knee'],
['right_knee', 'right_ankle'],
['left_knee', 'left_ankle'],
['right_ankle', 'right_heel', 'right_foot_index'],
['left_ankle', 'left_heel', 'left_foot_index']
]

segment_COM_lengths = [
.5,
.4310,
.5772,
.5772,
.4574,
.4574,
.7900, #check on the hand, did you actually use this in the com approx? Yes I did
.7900, #check on the hand, did you actually use this in the com approx? Yes I did
.4095,
.4095,
.4459,
.4459,
.4415, #check on the foot, did you actually use this in the com approx? No, I didn't
.4415  #check on the foot, did you actually use this in the com approx? No, I didn't
]

segment_COM_percentages = [
.0694,
.4346,
.0271,
.0271,
.0162,
.0162,
.0061,
.0061,
.1416,
.1416,
.0433,
.0433,
.0137,
.0137
]

num_segments = len(segments)

In [18]:

#create a table with the joint connections, COM length, and percentage for each segment into a dataframe
df = pd.DataFrame(list(zip(segments,joint_connections,segment_COM_lengths,segment_COM_percentages)),columns = ['Segment Name','Joint Connection','Segment COM Length','Segment COM Percentage'])

#create a new copy of this dataframe where the index is set to be the body segment (as opposed to 0,1,2 etc.)
segment_conn_len_perc_dataframe = df.set_index('Segment Name')


In [30]:
def build_mediapipe_skeleton(mediapipe_pose_data,segment_dataframe):
    """ This function takes in the mediapipe pose data array and the segment_conn_len_perc_dataframe. 
        For each frame of data, it loops through each segment we want to find and identifies the names
        of the proximal and distal joints of that segment. Then it searches the mediapipe_indices list
        to find the index that corresponds to the name of that segment. We plug the index into the 
        mediapipe_pose_data array to find the proximal/distal joints' XYZ coordinates at that frame. 
        The segment, its proximal joint and its distal joint gets thrown into a dictionary. 
        And then that dictionary gets saved to a list for each frame. By the end of the function, you 
        have a list that contains the skeleton segment XYZ coordinates for each frame."""
    
    #this is a list of the 33 pose joints that mediapipe tracks
    mediapipe_indices = [
    'nose',
    'right_eye_inner',
    'right_eye',
    'right_eye_outer',
    'left_eye_inner',
    'left_eye',
    'left_eye_outer',
    'right_ear',
    'left_ear',
    'mouth_right',
    'mouth_left',
    'right_shoulder',
    'left_shoulder',
    'right_elbow',
    'left_elbow',
    'right_wrist',
    'left_wrist',
    'right_pinky',
    'left_pinky',
    'right_index',
    'left_index',
    'right_thumb',
    'left_thumb',
    'right_hip',
    'left_hip',
    'right_knee',
    'left_knee',
    'right_ankle',
    'left_ankle',
    'right_heel',
    'left_heel',
    'right_foot_index',
    'left_foot_index'
    ]
  
    mediapipe_frame_segment_joint_XYZ = [] #empty list to hold all the skeleton XYZ coordinates/frame
    for frame in num_frame_range: #NOTE: need to change frame_range to numFrames 2/09/2022 - AC
        mediapipe_pose_skeleton_coordinates = {}
        for segment,segment_info in segment_conn_len_perc_dataframe.iterrows(): #iterate through the data frame by the segment name and all the info for that segment
            if segment == 'trunk':
                #assign joint connection to segment
                proximal_joint_left_shoulder = segment_info['Joint Connection'][0]
                proximal_joint_right_shoulder = segment_info['Joint Connection'][1]
                distal_joint_left_hip = segment_info['Joint Connection'][2]
                distal_joint_right_hip =segment_info['Joint Connection'][3]
                
                #assign index
                proximal_joint_left_shoulder_index = mediapipe_indices.index(proximal_joint_left_shoulder) 
                proximal_joint_right_shoulder_index = mediapipe_indices.index(proximal_joint_right_shoulder)
                distal_joint_left_hip_index = mediapipe_indices.index(distal_joint_left_hip)
                distal_joint_right_hip_index = mediapipe_indices.index(distal_joint_right_hip)
                
                #based on index, excract coordinate data from fmc mediapipe data
                mediapipe_pose_skeleton_coordinates[segment] = [mediapipe_pose_data[frame,proximal_joint_left_shoulder_index,:],
                                                                mediapipe_pose_data[frame,proximal_joint_right_shoulder_index,:], 
                                                                mediapipe_pose_data[frame,distal_joint_left_hip_index, :],
                                                                mediapipe_pose_data[frame,distal_joint_right_hip_index, :]
                                                                ]
            
            elif segment in ('left_hand', 'right_hand', 'right_foot','left_foot'): #segment == 'right_foot' or 'left_foot' or 'right_hand' or 'left_hand':
                #assign joint connectino to segment
                first_joint = segment_info['Joint Connection'][0]
                second_joint = segment_info['Joint Connection'][1]
                third_joint = segment_info['Joint Connection'][2]
                
                #assign index
                first_joint_index = mediapipe_indices.index(first_joint)
                second_joint_index = mediapipe_indices.index(second_joint)
                third_joint_index = mediapipe_indices.index(third_joint)
                
                #based on index, extract coordinate data from fmc mediapipe data
                mediapipe_pose_skeleton_coordinates[segment] = [mediapipe_pose_data[frame,first_joint_index, :],
                                                                mediapipe_pose_data[frame,second_joint_index, :],
                                                                mediapipe_pose_data[frame,third_joint_index, :]
                                                                ]
                
            else:
                proximal_joint_name = segment_info['Joint Connection'][0] 
                distal_joint_name = segment_info['Joint Connection'][1]

            #get the mediapipe index for the proximal and distal joint for this segment
                proximal_joint_index = mediapipe_indices.index(proximal_joint_name)
                distal_joint_index = mediapipe_indices.index(distal_joint_name)

            #use the mediapipe indices to get the XYZ coordinates for the prox/distal joints and throw it in a dictionary
            #mediapipe_pose_skeleton_coordinates[segment] = {'proximal':mediapipe_pose_data[frame,proximal_joint_index,:],'distal':mediapipe_pose_data[frame,distal_joint_index,:]}
                mediapipe_pose_skeleton_coordinates[segment] = [mediapipe_pose_data[frame,proximal_joint_index,:],mediapipe_pose_data[frame,distal_joint_index,:]]
                
        mediapipe_frame_segment_joint_XYZ.append(mediapipe_pose_skeleton_coordinates)
        f = 2 
    
    return mediapipe_frame_segment_joint_XYZ

    f= 2
skelcoordinates_frame_segment_joint_XYZ = build_mediapipe_skeleton(mediapipe_pose_data,segment_conn_len_perc_dataframe)
 
print(skelcoordinates_frame_segment_joint_XYZ[0])

f=2 

{'head': [array([nan, nan, nan]), array([nan, nan, nan])], 'trunk': [array([nan, nan, nan]), array([nan, nan, nan]), array([nan, nan, nan]), array([nan, nan, nan])], 'right_upper_arm': [array([nan, nan, nan]), array([nan, nan, nan])], 'left_upper_arm': [array([nan, nan, nan]), array([nan, nan, nan])], 'right_forearm': [array([nan, nan, nan]), array([nan, nan, nan])], 'left_forearm': [array([nan, nan, nan]), array([nan, nan, nan])], 'right_hand': [array([nan, nan, nan]), array([nan, nan, nan]), array([nan, nan, nan])], 'left_hand': [array([nan, nan, nan]), array([nan, nan, nan]), array([nan, nan, nan])], 'right_thigh': [array([nan, nan, nan]), array([nan, nan, nan])], 'left_thigh': [array([nan, nan, nan]), array([nan, nan, nan])], 'right_shin': [array([nan, nan, nan]), array([nan, nan, nan])], 'left_shin': [array([nan, nan, nan]), array([nan, nan, nan])], 'right_foot': [array([nan, nan, nan]), array([nan, nan, nan]), array([nan, nan, nan])], 'left_foot': [array([nan, nan, nan]), array([

In [37]:
def calculate_segment_COM(segment_conn_len_perc_dataframe,skelcoordinates_frame_segment_joint_XYZ):
    segment_COM_frame_dict = []
    for frame in num_frame_range:
        segment_COM_dict = {}
        for segment,segment_info in segment_conn_len_perc_dataframe.iterrows():
            this_segment_XYZ = skelcoordinates_frame_segment_joint_XYZ[frame][segment]
            
            if segment == 'trunk':
               trunk_proximal = (this_segment_XYZ[0] + this_segment_XYZ[1])/2
               trunk_distal = (this_segment_XYZ[2] + this_segment_XYZ[3])/2
               tunk_COM_length = segment_info['Segment COM Length']
               
               trunk_COM = this_segment_proximal + this_segment_COM_length*(this_segment_distal - this_segment_proximal)
               segment_COM_dict[segment] = trunk_COM
            
            elif segment in ('left_hand', 'right_hand'):
                wrist = this_segment_XYZ[0]
                index = this_segment_XYZ[1]
                pinky = this_segment_XYZ[2]
                knuckle_width = pinky - index
                third_metacarple = index + knuckle_width/3
                palm_segment = third_metacarple - wrist
                hand_COM_length = segment_info['Segment COM Length']
                
                hand_COM = wrist + hand_COM_length*palm_segment
                segment_COM_dict[segment] = hand_COM
                
                
            elif segment in ('left_foot', 'right_foot'):
                
                ankle = this_segment_XYZ[0]
                heel = this_segment_XYZ[1]
                foot_index = this_segment_XYZ[2]
                
                foot_COM = (ankle + heel + foot_index)/3
                segment_COM_dict[segment] = foot_COM
            else:
                #this_segment_proximal = this_segment_XYZ['proximal']
                #this_segment_distal = this_segment_XYZ['distal']
                this_segment_proximal = this_segment_XYZ[0]
                this_segment_distal = this_segment_XYZ[1]
                this_segment_COM_length = segment_info['Segment COM Length']

                this_segment_COM = this_segment_proximal + this_segment_COM_length*(this_segment_distal-this_segment_proximal)
                segment_COM_dict[segment] = this_segment_COM
        segment_COM_frame_dict.append(segment_COM_dict)
    return segment_COM_frame_dict
    f = 2


def reformat_segment_COM(segment_COM_frame_dict):
    
    segment_COM_frame_imgPoint_XYZ = np.empty([int(len(num_frame_range)),int(num_segments),3])
    for frame in num_frame_range:
        this_frame_skeleton = segment_COM_frame_dict[frame]
        for joint_count,segment in enumerate(this_frame_skeleton.keys()):
            segment_COM_frame_imgPoint_XYZ[frame,joint_count,:] = this_frame_skeleton[segment]
    return segment_COM_frame_imgPoint_XYZ


segment_COM_frame_dict = calculate_segment_COM(segment_conn_len_perc_dataframe, skelcoordinates_frame_segment_joint_XYZ)
segment_COM_frame_imgPoint_XYZ = reformat_segment_COM(segment_COM_frame_dict)

f=2

            



In [38]:
def calculate_total_body_COM(segment_conn_len_perc_dataframe,segment_COM_frame_dict):
    totalBodyCOM_frame_XYZ = np.empty([int(len(num_frame_range)),3])

    for frame in num_frame_range:

        this_frame_total_body_percentages = []
        this_frame_skeleton = segment_COM_frame_dict[frame]

        for segment, segment_info in segment_conn_len_perc_dataframe.iterrows():

            this_segment_COM = this_frame_skeleton[segment]
            this_segment_COM_percentage = segment_info['Segment COM Percentage']

            this_segment_total_body_percentage = this_segment_COM * this_segment_COM_percentage
            this_frame_total_body_percentages.append(this_segment_total_body_percentage)

        this_frame_total_body_COM = sum(this_frame_total_body_percentages)
        totalBodyCOM_frame_XYZ[frame,:] = this_frame_total_body_COM

    f=2
    return totalBodyCOM_frame_XYZ        
    
totalBodyCOM_frame_XYZ = calculate_total_body_COM(segment_conn_len_perc_dataframe,segment_COM_frame_dict)

f=2

In [40]:
good_frame = 6000
goodframe_x = skel_x[good_frame,:]
goodframe_y = skel_y[good_frame,:]
goodframe_z = skel_z[good_frame,:]

segment_COM_x = segment_COM_frame_imgPoint_XYZ[good_frame,:,0]
segment_COM_y = segment_COM_frame_imgPoint_XYZ[good_frame,:,1]
segment_COM_z = segment_COM_frame_imgPoint_XYZ[good_frame,:,2]

total_COM_x = totalBodyCOM_frame_XYZ[good_frame,0]
total_COM_y = totalBodyCOM_frame_XYZ[good_frame,1]
total_COM_z = totalBodyCOM_frame_XYZ[good_frame,2]



IndexError: index 6000 is out of bounds for axis 0 with size 391

def calculate_total_COM()

In [39]:
plot_this_frame = 6000


plot_frame_bones_XYZ = skelcoordinates_frame_segment_joint_XYZ[plot_this_frame]

%matplotlib
plt.ion
figure = plt.figure()
ax = figure.add_subplot(111, projection = '3d')

ax.scatter(goodframe_x, goodframe_y,goodframe_z)

#bonez
for segment in plot_frame_bones_XYZ.keys():
    prox_joint = plot_frame_bones_XYZ[segment][0] 
    dist_joint = plot_frame_bones_XYZ[segment][1]
    
    bone_x,bone_y,bone_z = [prox_joint[0],dist_joint[0]],[prox_joint[1],dist_joint[1]],[prox_joint[2],dist_joint[2]] 

    ax.plot(bone_x,bone_y,bone_z,color = 'black')
    
ax.scatter(segment_COM_x,segment_COM_y,segment_COM_z, color = 'orange')    

ax.scatter(total_COM_x,total_COM_y,total_COM_z, color = 'purple')
plt.show()
f = 2







IndexError: list index out of range

In [ ]:
# %matplotlib
# #plt.ion()
# figure = plt.figure()
# ax = figure.add_subplot(111, projection = '3d')
# ax.scatter(goodframe_x, goodframe_y,goodframe_z)
# ax.scatter(segment_COM_x,segment_COM_y,segment_COM_z)
# plt.show()
# # inspect(ax,methods= True)
# f=2

Using matplotlib backend: <object object at 0x00000256B7FC4C90>


In [ ]:
# #http://www.kdm.p.lodz.pl/articles/2017/3/21_3_4.pdf
# r_upperarm_COM_x = goodframe_x[r_upperarm[0]] + (.5772)*(goodframe_x[r_upperarm[1]]-goodframe_x[r_upperarm[0]])
# l_upperarm_COM_x = goodframe_x[l_upperarm[0]] + (.5772)*(goodframe_x[l_upperarm[1]]-goodframe_x[l_upperarm[0]])

# r_forearm_COM_x = goodframe_x[r_forearm[0]] + (.4574)*(goodframe_x[r_forearm[1]]-goodframe_x[r_forearm[0]])
# l_forearm_COM_x = goodframe_x[l_forearm[0]] + (.4574)*(goodframe_x[l_forearm[1]]-goodframe_x[l_forearm[0]])

# r_upperarm_COM_y = goodframe_y[r_upperarm[0]] + (.5772)*(goodframe_y[r_upperarm[1]]-goodframe_y[r_upperarm[0]])
# l_upperarm_COM_y = goodframe_y[l_upperarm[0]] + (.5772)*(goodframe_y[l_upperarm[1]]-goodframe_y[l_upperarm[0]])

# r_forearm_COM_y = goodframe_y[r_forearm[0]] + (.4574)*(goodframe_y[r_forearm[1]]-goodframe_y[r_forearm[0]])
# l_forearm_COM_y = goodframe_y[l_forearm[0]] + (.4574)*(goodframe_y[l_forearm[1]]-goodframe_y[l_forearm[0]])

# segment_com_x = [r_upperarm_COM_x,l_upperarm_COM_x,r_forearm_COM_x,l_forearm_COM_x]
# segment_com_y = [r_upperarm_COM_y,l_upperarm_COM_y,r_forearm_COM_y,l_forearm_COM_y]

